# GLD to NEET

In [ ]:
library(bigrquery)
library(tidyverse)
library(dbplyr)
library(lavaan)
library(cowplot)
library(margins)
library(finalfit)

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"
dataset = "CB_FDM_DepartmentForEducation"

# Store the FDM targetdb
targetdb <- paste(project_id, dataset, sep = ".")
targetdb <-gsub(' ','',targetdb)
#print (targetdb)

# create connection to database
con <- DBI::dbConnect(bigrquery::bigquery(), 
                      project = project_id,
                      dataset = dataset)

print(paste0("Connected to : ", targetdb))

In [ ]:
my_theme <- function() {
  theme_cowplot() %+replace%
    theme(strip.background = element_blank(),
          strip.placement = "outside",
          panel.spacing = unit(1, "lines"),
          text = element_text(size = 9),
          strip.text = element_text(size = 9),
          axis.text = element_text(size = 7),
          axis.title = element_text(size = 9),
          legend.position = "none") 
}

## EYFSP

In [ ]:
# people with a single EYFSP entry
all_eyfsp <- tbl(con, "src_EYFSP")

eyfsp_students_multiple_entries <- all_eyfsp |>
    select(person_id, AcademicYear) |>
    group_by(person_id, AcademicYear) |>
    summarise(n_per_year = n()) |>
    group_by(person_id) |>
    mutate(n_overall = n()) |> 
    filter(n_overall >= 2) |>
    select(-c(n_overall,n_per_year)) |>
    ungroup() |>
    arrange(person_id) |> 
    pull(person_id) |> 
    unique() 

length(eyfsp_students_multiple_entries)

In [ ]:
eyfsp_students <- all_eyfsp |>
    group_by(person_id, AcademicYear) |>
    summarise(n = n()) |>
    group_by(person_id) |>
    filter(n == 1) |>
    select(-n) |>
    mutate(in_eyfsp = TRUE) |>
    rename(eyfsp_AcademicYear = AcademicYear)

eyfsp_students <- eyfsp_students |>
    filter(!eyfsp_AcademicYear %in% c("2002/2003", "2003/2004", "2004/2005", "2005/2006"))

In [ ]:
# calculate GLD for the require years
gld_2007_to_2012 <- all_eyfsp |>
    left_join(eyfsp_students) |>
    filter(!is.na(in_eyfsp)) |>
    filter(eyfsp_AcademicYear %in% c("2006/2007", "2007/2008", "2008/2009", "2009/2010", "2010/2011", "2011/2012"),
           !is.na(EYFSPTotal)) |>
    mutate(GLD_new = case_when(
        as.numeric(EYFSPTotal) >= 78 & 
            PSEAS1 != "N" & as.numeric(PSEAS1) >= 6 & 
            PSEAS2 != "N" & as.numeric(PSEAS2) >= 6 & 
            PSEAS3 != "N" & as.numeric(PSEAS3) >= 6 & 
            CLLAS1 != "N" & as.numeric(CLLAS1) >= 6 & 
            CLLAS2 != "N" & as.numeric(CLLAS2) >= 6 & 
            CLLAS3 != "N" & as.numeric(CLLAS3) >= 6 & 
            CLLAS4 != "N" & as.numeric(CLLAS4) >= 6 ~ TRUE,
        TRUE ~ FALSE
    )) |> 
    select(person_id, eyfsp_AcademicYear, GLD_new)

In [ ]:
# compare new gld to old for 2011/2012
eyfsp_gld <- all_eyfsp |>
    left_join(eyfsp_students) |>
    filter(!is.na(in_eyfsp)) |> 
    left_join(gld_2007_to_2012, by = c("person_id", "eyfsp_AcademicYear")) |> 
    filter(!(is.na(GLD) & is.na(GLD_new)))

eyfsp_gld_2012 <- eyfsp_gld |> 
    filter(eyfsp_AcademicYear == "2011/2012") |> 
    select(GLD, GLD_new) |> 
    collect()

table(eyfsp_gld_2012$GLD, eyfsp_gld_2012$GLD_new)

In [ ]:
eyfsp <- eyfsp_gld |> 
    filter(!person_id %in% eyfsp_students_multiple_entries) |> 
    mutate(GLD_final = coalesce(GLD_new, GLD),
           gld_dummy = ifelse(GLD_final, 1, 0),
           in_eyfsp = TRUE)

## NCCIS w/ EYFSP

In [ ]:
# check how many valid people with EYFSP entries have NEET status
nccis_with_eyfsp_temp <- tbl(con, "src_NCCIS") |>
    left_join(
        eyfsp |>
            select(person_id, eyfsp_AcademicYear, in_eyfsp, GLD_final, gld_dummy)
    ) |>
    filter(!is.na(in_eyfsp)) 

unknowns <- nccis_with_eyfsp_temp |> 
    group_by(person_id) |> 
    summarise(unknown = all(CurrentActivityCode %in% c(810, 820, 830))) |> 
    filter(unknown) |> 
    pull(person_id)

nccis_with_eyfsp <- nccis_with_eyfsp_temp |> 
    filter(!person_id %in% unknowns)

In [ ]:
person_ids_in_2007 <- nccis_with_eyfsp |> 
    select(person_id, AcademicYear, eyfsp_AcademicYear) |>
    mutate(eyfsp_AcademicYearStart = substr(eyfsp_AcademicYear, 1, 4),
           eyfsp_AcademicYearEnd = substr(eyfsp_AcademicYear, 6, 9)) |>
    filter(eyfsp_AcademicYearEnd == "2007") |> 
    pull(person_id) |> 
    unique()

length(person_ids_in_2007)

nccis_with_eyfsp <- nccis_with_eyfsp |> 
    filter(person_id %in% person_ids_in_2007)

In [ ]:
ever_neet_gld <- nccis_with_eyfsp |> 
    group_by(person_id, GLD_final, gld_dummy) |> 
    summarise(ever_neet = any(!is.na(NEETStartDate))) |> 
    mutate(ever_neet_dummy = ifelse(ever_neet, 1, 0))

nrow(ever_neet_gld |> collect())
length(ever_neet_gld |> pull(person_id) |> unique())

## GLD -> NEET with covariates

In [ ]:
people_in_final_sample <- ever_neet_gld |> 
    ungroup() |> 
    pull(person_id) |> 
    unique() 

length(people_in_final_sample)

### Person table

In [ ]:
master_con <- DBI::dbConnect(bigrquery::bigquery(), 
                      project = project_id,
                      dataset = "CB_FDM_MASTER")

person_table <- tbl(master_con, "person") |> 
    filter(person_id %in% people_in_final_sample) |> 
    select(person_id, month_of_birth, gender_source_value, ethnicity_source_value) |> 
    mutate(academic_month_of_birth = ifelse(month_of_birth - 9 < 0, month_of_birth - 9 + 12, month_of_birth - 9),
           gender = ifelse(gender_source_value == "U", NA, gender_source_value))

#### Ethnicity (preliminary)

In [ ]:
person_table_final <- person_table |> 
    mutate(ethnicity = case_when(
        str_detect(ethnicity_source_value, "Unknown") ~ NA_character_,
        str_detect(ethnicity_source_value, "Asian or Asian British: Indian") ~ "South Asian",
        str_detect(ethnicity_source_value, "Asian or Asian British: Pakistani") ~ "South Asian",
        str_detect(ethnicity_source_value, "Asian or Asian British: Bangladeshi") ~ "South Asian",
        str_detect(ethnicity_source_value, "White: English or Welsh or Scottish or Northern Irish or British") ~ "White British",
        TRUE ~ "Other"
    )) |> 
    collect()

#### Month of birth

In [ ]:
month_of_birth <- person_table_final |> 
    select(person_id, academic_month_of_birth)

#### Gender

In [ ]:
gender <- person_table_final |> 
    mutate(Male = ifelse(is.na(gender), NA, ifelse(gender == "M", 1, 0))) |> 
    select(person_id, Male)

#### Combined ethnicity from person table and school census

In [ ]:
school_census <- tbl(con, "src_census") |> 
    filter(person_id %in% people_in_final_sample) |> 
    select(person_id, Ethnicity) |> 
    collect()

school_based_ethnicity <- school_census |> 
    arrange(person_id) |> 
    group_by(person_id, Ethnicity) |> 
    summarise(n = n()) |> 
    group_by(person_id) |> 
    filter(!is.na(Ethnicity), !Ethnicity %in% c("NOBT", "UNCL", "REFU")) |> 
    filter(Ethnicity == Ethnicity[first(which(n == max(n)))]) |> 
    mutate(EthnicityGroup = case_when(
        is.na(Ethnicity) ~ NA_character_,
        Ethnicity %in% c("NOBT", "UNCL", "REFU") ~ NA_character_,
        Ethnicity %in% c("ABAN", "AIND", "AKAO", "AKPA", "AMPK", "AOPK", "APKN") ~ "South Asian",
        Ethnicity %in% c("WBRI", "WENG", "WCOR", "WSCO", "WOWB", "WWEL", "WIRI") ~ "White British",
        TRUE ~ "Other"
    ))

combined_ethnicity <- person_table_final |> 
    rename(person_ethnicity = ethnicity) |> 
    left_join(school_based_ethnicity) |> 
    mutate(CombinedEthnicity = coalesce(person_ethnicity, EthnicityGroup))

combined_ethnicity |> 
    group_by(CombinedEthnicity) |> 
    summarise(n = n())

combined_ethnicity |> 
    ungroup() |> 
    filter(!is.na(person_ethnicity) & !is.na(EthnicityGroup)) |> 
    mutate(match = person_ethnicity == EthnicityGroup) |> 
    group_by(person_ethnicity) |> 
    summarise(prop_match = sum(match) / n(),
              sum_match = sum(match),
             n = n())

ethnicity <- combined_ethnicity |> 
    select(person_id, CombinedEthnicity) |> 
    mutate(EthnicityAsian = ifelse(is.na(CombinedEthnicity), NA, ifelse(CombinedEthnicity == "South Asian", 1, 0)),
           EthnicityOther = ifelse(is.na(CombinedEthnicity), NA, ifelse(CombinedEthnicity == "Other", 1, 0)),
           CombinedEthnicity = factor(CombinedEthnicity, levels = c("White British", "South Asian", "Other")))

#### English as additional language (EAL)

In [ ]:
language_temp <- tbl(con, "src_census") |> 
    filter(person_id %in% people_in_final_sample) |> 
    select(person_id, Language) |> 
    collect()

english_additional_language <- language_temp |> 
    filter(!is.na(Language)) |> 
    mutate(EAL = !Language %in% c("ENG", "ENB")) |> 
    group_by(person_id, EAL) |> 
    summarise(n = n()) |> 
    group_by(person_id) |> 
    slice_max(n) |> 
    mutate(num = n()) |> 
    filter(num == 1 | num > 1 & EAL) |> 
    select(person_id, EAL) |> 
    mutate(EAL = ifelse(EAL, 1, 0))

#### Ever Free School Meals (eFMS)

In [ ]:
years_to_include <- paste0(seq(2006, 2017, 1), "/", seq(2007, 2018, 1))

fsm_temp <- tbl(con, "src_census") |> 
    filter(person_id %in% people_in_final_sample,
           AcademicYear %in% years_to_include) |> 
    select(person_id, FSMEligible, AcademicYear) |> 
    collect()

fsm <- fsm_temp |> 
    filter(!is.na(FSMEligible)) |> 
    group_by(person_id) |> 
    summarise(ever_fsm = any(FSMEligible))

#### Ever SEN

In [ ]:
sen_temp <- tbl(con, "src_census") |> 
    filter(person_id %in% people_in_final_sample,
          AcademicYear %in% years_to_include) |> 
    select(person_id, SENprovision, AcademicYear) |> 
    collect()

sen <- sen_temp |> 
    filter(!is.na(SENprovision)) |> 
    group_by(person_id) |> 
    summarise(ever_sen = any(SENprovision %in% c("A", "P", "S", "K", "E")))

## Make full dataset

In [ ]:
ever_neet_gld_cov <- ever_neet_gld |> 
    collect() |> 
    left_join(gender) |> 
    left_join(month_of_birth) |> 
    left_join(ethnicity) |> 
    left_join(english_additional_language) |> 
    left_join(fsm) |> 
    left_join(sen) |> 
    filter(!is.na(CombinedEthnicity), !is.na(Male), !is.na(EAL), !is.na(ever_fsm), !is.na(ever_sen))

nrow(ever_neet_gld_cov)

summary_factorlist(ever_neet_gld_cov, explanatory = c("ever_neet", "GLD_final", "ever_fsm", "ever_sen", "Male", "academic_month_of_birth", "EAL", "CombinedEthnicity"))

### Model 1

In [ ]:
summary(neet_gld_model_probit <- glm(ever_neet ~ GLD_final, data = ever_neet_gld_cov, family = binomial(link = "probit")))

confint(neet_gld_model_probit)
DescTools::PseudoR2(neet_gld_model_probit, which = "McFadden")
DescTools::PseudoR2(neet_gld_model_probit, which = "McKelveyZavoina")

In [ ]:
print(paste0("Probability of NEET without reaching GLD: ", sprintf("%.3f", pnorm(coef(neet_gld_model_probit)[[1]]))))
print(paste0("Probability of NEET with reaching GLD: ", sprintf("%.3f", pnorm(coef(neet_gld_model_probit)[[1]] + coef(neet_gld_model_probit)[[2]]))))

In [ ]:
ever_neet_gld_matrix <- ever_neet_gld_cov |> 
    group_by(ever_neet, GLD_final) |> 
    summarise(n = n())

ever_neet_gld_matrix |> 
    ggplot(aes(x = GLD_final, y = ever_neet, fill = n)) +
        geom_tile() +
        geom_text(aes(label = paste0("n = ", n)), color = "white", fontface = "bold", size = 12 * .36) +
        labs(x = "Reached GLD", y = "Ever NEET") +
        cowplot::theme_cowplot() +
        theme(legend.position = "none") +
        coord_equal()

### Model 2

Demographic table:

In [ ]:
summary(neet_gld_cov_model_probit <- glm(ever_neet ~ GLD_final + ever_fsm + ever_sen + Male + academic_month_of_birth + EAL + CombinedEthnicity, data = ever_neet_gld_cov, family = binomial(link = "probit")))
confint(neet_gld_cov_model_probit)

library(margins)

summary(margins(neet_gld_cov_model_probit))
DescTools::PseudoR2(neet_gld_cov_model_probit, which = "McFadden")
DescTools::PseudoR2(neet_gld_cov_model_probit, which = "McKelveyZavoina")

In [ ]:
people <- data.frame(academic_month_of_birth = c(11, 0),
                     CombinedEthnicity = c("White British", "Other", "White British", "Other"),
                     EAL = c(0, 1, 0, 1),
                     ever_fsm = c(FALSE, TRUE, FALSE, TRUE),
                    ever_sen = c(FALSE, TRUE, FALSE, TRUE),
                    GLD_final = c(TRUE, FALSE, FALSE, TRUE),
                     Male = c(0, 1, 0, 1)) %>%
    mutate(prob_neet = predict(neet_gld_cov_model_probit, newdata = ., type = "response"),
          group = c("least", "most", "least_no_gld", "most_with_gld"))

people |> 
    left_join(ever_neet_gld_cov) |> 
    group_by(group) |> 
    summarise(n = n(),
             prob_neet = first(prob_neet))

## Do key stages mediate GLD -> NEET?

### Key stage 1

In [ ]:
ks1_full_all <- tbl(con, "src_KS1") |>
    select(person_id, AcademicYear, Level2Reading, Level2Writing, Level2Maths, Level2Science, Reading, Writing, Maths, Science) |> 
    mutate(ks1_expected_3 = Level2Reading & Level2Writing & Level2Maths,
           ks1_AcademicYear = AcademicYear)

ks1_full <- ks1_full_all |> 
    filter(!is.na(ks1_expected_3)) |> 
    mutate(ks1_ReadingCat = Reading, ks1_WritingCat = Writing, ks1_MathsCat = Maths, ks1_ScienceCat = Science) |> 
    mutate(across(ks1_ReadingCat:ks1_ScienceCat, ~ case_when(. %in% c("A", "D", "IN", "U") ~ NA_real_,
                                                             . == "W" ~ 0,
                                                             . == "1" ~ 1,
                                                             . %in% c("2", "2C", "2B", "2A") ~ 2,
                                                             . == "3" ~ 3,
                                                             . %in% c("4", "4+") ~ 4,
                                                             . == "5" ~ 5,
                                                             . == "6" ~ 6, 
                                                             TRUE ~ NA_real_))) |> 
    mutate(ks1_ReadingBinary = ks1_ReadingCat, ks1_WritingBinary = ks1_WritingCat, ks1_MathsBinary = ks1_MathsCat, ks1_ScienceBinary = ks1_ScienceCat) |> 
    mutate(across(ks1_ReadingBinary:ks1_ScienceBinary, ~ ifelse(is.na(.), NA, ifelse(. >= 2, TRUE, FALSE)))) |> 
    select(person_id, ks1_AcademicYear, ks1_ReadingCat:ks1_ScienceCat, ks1_ReadingBinary:ks1_ScienceBinary) |> 
    mutate(ks1_Expected3 = ifelse(is.na(ks1_ReadingBinary) | is.na(ks1_WritingBinary) | is.na(ks1_MathsBinary), 
                                  NA, 
                                  ks1_ReadingBinary & ks1_WritingBinary & ks1_MathsBinary),
           ks1_Expected4 = ifelse(is.na(ks1_ReadingBinary) | is.na(ks1_WritingBinary) | is.na(ks1_MathsBinary) | is.na(ks1_ScienceBinary), 
                                  NA, 
                                  ks1_ReadingBinary & ks1_WritingBinary & ks1_MathsBinary & ks1_ScienceBinary))

# multiple entries?
ks1_students_multiple_entries <- ks1_full |> 
    group_by(person_id) |> 
    summarise(n = n()) |> 
    filter(n >= 2) |> 
    pull(person_id)

length(ks1_students_multiple_entries)

# remove multiple entries
ks1_full <- ks1_full |> 
    filter(!person_id %in% ks1_students_multiple_entries)

### Key Stage 2

In [ ]:
ks2_full <- tbl(con, "src_KS2_pupil") |>
    select(person_id, AcademicYear, ReadingLevel, WritingLevel, MathsLevel, GPSLevel, Level4Reading, Level4Writing, Level4Maths, Level4GPS) |> 
    mutate(WritingLevel = ifelse(is.na(WritingLevel), GPSLevel, WritingLevel),
           Level4Writing = ifelse(is.na(Level4Writing), Level4GPS, Level4Writing),
           ks2_AcademicYear = AcademicYear,
           ks2_expected_3 = ifelse(is.na(Level4Reading) | is.na(Level4Writing) | is.na(Level4Maths), NA, Level4Reading & Level4Writing & Level4Maths)) |>  
    filter(!is.na(ks2_expected_3)) |> 
    mutate(ks2_ReadingCat = ReadingLevel, ks2_WritingCat = WritingLevel, ks2_MathsCat = MathsLevel) |> 
    mutate(across(ks2_ReadingCat:ks2_MathsCat, ~ case_when(. %in% c("A", "M", "Q", "S", "T", "X", "Z", "N") ~ NA_real_,
                                                           . %in% c("B") ~ 0,
                                                           . == "1" ~ 1,
                                                           . %in% c("2", "2C", "2B", "2A") ~ 2,
                                                           . == "3" ~ 3,
                                                           . %in% c("4", "4+", "4C", "4B", "4A") ~ 4,
                                                           . == "5" ~ 5,
                                                           . == "6" ~ 6, 
                                                           TRUE ~ NA_real_))) |> 
    mutate(ks2_ReadingBinary = ks2_ReadingCat, ks2_WritingBinary = ks2_WritingCat, ks2_MathsBinary = ks2_MathsCat) |> 
    mutate(across(ks2_ReadingBinary:ks2_MathsBinary, ~ ifelse(is.na(.), NA, ifelse(. >= 4, TRUE, FALSE)))) |> 
    select(person_id, ks2_AcademicYear, ks2_ReadingCat:ks2_MathsCat, ks2_ReadingBinary:ks2_MathsBinary) |> 
    mutate(ks2_Expected3 = ifelse(is.na(ks2_ReadingBinary) | is.na(ks2_WritingBinary) | is.na(ks2_MathsBinary), NA, ks2_ReadingBinary & ks2_WritingBinary & ks2_MathsBinary))

# multiple entries?
ks2_students_multiple_entries <- ks2_full |> 
    group_by(person_id) |> 
    summarise(n = n()) |> 
    filter(n >= 2) |> 
    pull(person_id)

length(ks2_students_multiple_entries)

# remove multiple entries
ks2_full <- ks2_full |> 
    filter(!person_id %in% ks2_students_multiple_entries)

### Key Stage 4

In [ ]:
ks4_full <- tbl(con, "src_KS4_pupil") |>
    select(person_id, AcademicYear, Entered5, FiveLevel2, FiveLevel2EM, MathsPass, EnglishPass, SciencePass, EnglishAttempt, MathsAttempt, ScienceAttempt, Attainment8, A8Maths, A8English) |> 
    mutate(ks4_Expected = ifelse(Entered5, FiveLevel2, NA),
           ks4_MathsPass = ifelse(MathsAttempt, MathsPass, NA),
           ks4_EnglishPass = ifelse(EnglishAttempt, EnglishPass, NA),
           ks4_SciencePass = ifelse(ScienceAttempt, SciencePass, NA),
           ks4_AcademicYear = AcademicYear,
           ks4_Entered5EM = Entered5 & EnglishAttempt & MathsAttempt,
           ks4_ExpectedEM = ifelse(ks4_Entered5EM, FiveLevel2EM, NA)) |> 
    select(person_id, ks4_AcademicYear, ks4_MathsPass, ks4_EnglishPass, ks4_Expected, ks4_ExpectedEM, ks4_SciencePass)

bad_AcademicYears <- paste(seq("2001", "2011"), seq("2002", "2012"), sep = "/")

# multiple entries?
ks4_students_multiple_entries <- ks4_full |> 
    group_by(person_id) |> 
    summarise(n = n()) |> 
    filter(n >= 2) |> 
    pull(person_id)

# remove multiple entries
ks4_full <- ks4_full |> 
    filter(!person_id %in% ks4_students_multiple_entries)

## Make full dataset

In [ ]:
ever_neet_gld_ks1_ks2_ks4 <- ever_neet_gld |> 
    left_join(
        ks1_full |>
            mutate(in_ks1 = TRUE),
        by = "person_id"
    ) |> 
    left_join(
        ks2_full |>
            mutate(in_ks2 = TRUE),
        by = "person_id"
    ) |> 
    left_join(
        ks4_full |>
            mutate(in_ks4 = TRUE),
        by = "person_id"
    ) |> 
    collect()

ever_neet_gld_ks1_ks2_ks4_covariates <- ever_neet_gld_ks1_ks2_ks4 |> 
    collect() |> 
    left_join(gender) |> 
    left_join(month_of_birth) |> 
    left_join(ethnicity) |> 
    left_join(english_additional_language) |> 
    left_join(fsm) |> 
    left_join(sen) |> 
    mutate(across(c(ever_fsm, ever_sen), as.numeric)) |> 
    filter(!is.na(CombinedEthnicity), !is.na(Male), !is.na(EAL), !is.na(ever_fsm), !is.na(ever_sen))

nrow(ever_neet_gld_ks1_ks2_ks4_covariates)

### Academic missingness

#### KS1

In [ ]:
ever_neet_gld_ks1_ks2_ks4_covariates |> 
    group_by(ks1_ReadingBinary, ks1_WritingBinary, ks1_MathsBinary) |> 
    rowwise() |> 
    mutate(any_passed = any(!is.na(ks1_ReadingBinary) & ks1_ReadingBinary, !is.na(ks1_WritingBinary) & ks1_WritingBinary, !is.na(ks1_MathsBinary) & ks1_MathsBinary),
           any_na = any(is.na(ks1_ReadingBinary), is.na(ks1_WritingBinary), is.na(ks1_MathsBinary)),
           num_na = sum(is.na(ks1_ReadingBinary), is.na(ks1_WritingBinary), is.na(ks1_MathsBinary))) |> 
    group_by(any_passed, any_na, num_na) |> 
    summarise(n = n())

# not in dataset at all
sum(is.na(ever_neet_gld_ks1_ks2_ks4_covariates$ks1_AcademicYear))

#### KS2

In [ ]:
ever_neet_gld_ks1_ks2_ks4_covariates |> 
    group_by(ks2_ReadingBinary, ks2_WritingBinary, ks2_MathsBinary) |> 
    rowwise() |> 
    mutate(any_passed = any(!is.na(ks2_ReadingBinary) & ks2_ReadingBinary, !is.na(ks2_WritingBinary) & ks2_WritingBinary, !is.na(ks2_MathsBinary) & ks2_MathsBinary),
           any_na = any(is.na(ks2_ReadingBinary), is.na(ks2_WritingBinary), is.na(ks2_MathsBinary)),
           num_na = sum(is.na(ks2_ReadingBinary), is.na(ks2_WritingBinary), is.na(ks2_MathsBinary))) |> 
    group_by(any_passed, any_na, num_na) |> 
    summarise(n = n())

sum(is.na(ever_neet_gld_ks1_ks2_ks4_covariates$ks2_AcademicYear))

#### KS4

In [ ]:
ever_neet_gld_ks1_ks2_ks4_covariates |> 
    group_by(ks4_MathsPass, ks4_EnglishPass, ks4_Expected) |> 
    rowwise() |> 
    mutate(any_passed = any(!is.na(ks4_MathsPass) & ks4_MathsPass, !is.na(ks4_EnglishPass) & ks4_EnglishPass, !is.na(ks4_Expected) & ks4_Expected),
           any_na = any(is.na(ks4_MathsPass), is.na(ks4_EnglishPass), is.na(ks4_Expected)),
           num_na = sum(is.na(ks4_MathsPass), is.na(ks4_EnglishPass), is.na(ks4_Expected))) |> 
    group_by(any_passed, any_na, num_na) |> 
    summarise(n = n())

sum(is.na(ever_neet_gld_ks1_ks2_ks4_covariates$ks4_AcademicYear))

### Model 3

In [ ]:
gld_ks1_ks2_ks4_neet_direct_expected_eq <- '
    ks1 ~ a*gld_dummy
    ks2 ~ b*ks1
    ks4 ~ c*ks2
    ever_neet ~ d*gld_dummy + e*ks4

    ks1 =~ ks1_WritingCat + ks1_ReadingCat + ks1_MathsCat
    ks2 =~ ks2_WritingCat + ks2_ReadingCat + ks2_MathsCat
    ks4 =~ ks4_MathsPass + ks4_EnglishPass + ks4_Expected

    gld_direct := d
    gld_indirect := a*b*c*e
    total := gld_direct + gld_indirect
'

gld_ks1_ks2_ks4_neet_direct_expected_fit <- sem(gld_ks1_ks2_ks4_neet_direct_expected_eq, ever_neet_gld_ks1_ks2_ks4_covariates, 
   ordered = c("ks1_WritingCat", "ks1_ReadingCat", "ks1_MathsCat", "ks2_WritingCat", "ks2_ReadingCat", "ks2_MathsCat", "ks4_MathsPass", "ks4_EnglishPass", "ks4_Expected", "ever_neet"),
   missing = "pairwise")

print(summary(gld_ks1_ks2_ks4_neet_direct_expected_fit, standardized=TRUE))
parameterEstimates(gld_ks1_ks2_ks4_neet_direct_expected_fit, ci = TRUE)
fitMeasures(gld_ks1_ks2_ks4_neet_direct_expected_fit, c("chisq", "df", "pvalue", "cfi", "rmsea"))

### Model 4

In [ ]:
gld_ks1_ks2_ks4_neet_direct_expected_cov_eq <- '
    ks1 ~ a*gld_dummy + EthnicityAsian + EthnicityOther + EAL + Male + academic_month_of_birth + ever_fsm + ever_sen
    ks2 ~ b*ks1 + EthnicityAsian + EthnicityOther + EAL + Male + academic_month_of_birth + ever_fsm + ever_sen
    ks4 ~ c*ks2 + EthnicityAsian + EthnicityOther + EAL + Male + academic_month_of_birth + ever_fsm + ever_sen
    ever_neet ~ d*gld_dummy + e*ks4 + EthnicityAsian + EthnicityOther + EAL + Male + academic_month_of_birth + ever_fsm + ever_sen

    ks1 =~ ks1_WritingCat + ks1_ReadingCat + ks1_MathsCat
    ks2 =~ ks2_WritingCat + ks2_ReadingCat + ks2_MathsCat
    ks4 =~ ks4_MathsPass + ks4_EnglishPass + ks4_Expected

    gld_direct := d
    gld_indirect := a*b*c*e
    total := gld_direct + gld_indirect
'

gld_ks1_ks2_ks4_neet_direct_expected_cov_fit <- sem(gld_ks1_ks2_ks4_neet_direct_expected_cov_eq, ever_neet_gld_ks1_ks2_ks4_covariates, 
   ordered = c("ks1_WritingCat", "ks1_ReadingCat", "ks1_MathsCat", "ks2_WritingCat", "ks2_ReadingCat", "ks2_MathsCat", "ks4_MathsPass", "ks4_EnglishPass", "ks4_Expected", "ever_neet"),
   missing = "pairwise")

print(summary(gld_ks1_ks2_ks4_neet_direct_expected_cov_fit, standardized=TRUE))
parameterEstimates(gld_ks1_ks2_ks4_neet_direct_expected_cov_fit, ci = TRUE)
fitMeasures(gld_ks1_ks2_ks4_neet_direct_expected_cov_fit, c("chisq", "df", "pvalue", "cfi", "rmsea"))

### Combined Plot

In [ ]:
model1_coefs <- broom::tidy(neet_gld_model_probit, conf.int = TRUE)[2, ] |> 
    mutate(model = "Model 1", path = "GLD -> NEET (Total)") |> 
    select(model, path, estimate, conf.low, conf.high, p.value)
model2_coefs <- broom::tidy(neet_gld_cov_model_probit, conf.int = TRUE)[2, ] |> 
    mutate(model = "Model 2", path = "GLD -> NEET (Total)") |> 
    select(model, path, estimate, conf.low, conf.high, p.value)
model3_coefs <- broom::tidy(gld_ks1_ks2_ks4_neet_direct_expected_fit, conf.int = TRUE)[c(1:3, 5, 82, 83), ] |> 
    mutate(model = "Model 3", path = c("GLD -> KS1", "KS1 -> KS2", "KS2 -> KS4", "KS4-> NEET", "GLD -> NEET (Direct)", "GLD -> NEET (Indirect)")) |> 
    select(model, path, estimate, conf.low, conf.high, p.value)
model4_coefs <- broom::tidy(gld_ks1_ks2_ks4_neet_direct_expected_cov_fit, conf.int = TRUE)[c(1, 9, 17, 26, 152, 153), ] |> 
    mutate(model = "Model 4", path = c("GLD -> KS1", "KS1 -> KS2", "KS2 -> KS4", "KS4-> NEET", "GLD -> NEET (Direct)", "GLD -> NEET (Indirect)")) |> 
    select(model, path, estimate, conf.low, conf.high, p.value)

all_coefs <- rbind(model1_coefs, model2_coefs, model3_coefs, model4_coefs) |> 
    mutate(path = factor(path, levels = c("GLD -> NEET (Total)", "GLD -> NEET (Direct)", "GLD -> NEET (Indirect)", "GLD -> KS1", "KS1 -> KS2", "KS2 -> KS4", "KS4-> NEET"))) |> 
    arrange(model, path)
all_coefs 

In [ ]:
model_colors <- PNWColors::pnw_palette("Bay", 4, type = "discrete")

coefficient_plot <- ggplot(all_coefs, aes(x = estimate, y = path, color = model)) +
    geom_vline(xintercept = 0, linetype = "dotted", color = "gray50") +
    ggstance::geom_crossbarh(aes(xmin = conf.low, xmax = conf.high, fill = model), width = 0.5, size = 0.5, alpha = 0.2, fatten = 1.5, position = ggstance::position_dodgev(height = -0.7), color = NA) +
    ggstance::geom_crossbarh(aes(xmin = conf.low, xmax = conf.high, fill = model), width = 0.5, alpha = 0, size = 0.05, fatten = 10, position = ggstance::position_dodgev(height = -0.7)) +
    scale_y_discrete(limits = rev) +
    cowplot::theme_cowplot() +
    labs(x = "Estimate", y = "") +
    theme(axis.line.y = element_blank(),
          axis.ticks.y = element_blank(),
          axis.text = element_text(size = 7),
          axis.title = element_text(size = 9),
          legend.text = element_text(size = 7),
          legend.title = element_blank(),
          legend.position = c(.95, .95),
          legend.justification = c("right", "top"),
          axis.title.y = element_blank()) +
    scale_x_continuous(limits = c(-1, 1.5), breaks = seq(-1, 1.5, 0.5)) +
    scale_color_manual(values = model_colors) +
    scale_fill_manual(values = model_colors)
coefficient_plot

In [ ]:
ggsave(file="coefficients.svg", width=3, height=3, device = "svg", fix_text_size = FALSE)